# Imports packages and assign variables

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid, RadiusNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
import xgboost as xgb
import time
file_paths = [
    'testData/1.10_A_30s_600ms_3/',
    'testData/2.50_A_30s_600ms_3/',
    'testData/3.90_A_30s_600ms_3/',
    'testData/4.130_A_30s_600ms_3/',
    'testData/5.60_A_30s_200ms_3/',
    'testData/6.60_A_30s_500ms_3/',
    'testData/7.60_A_30s_800ms_3/',
    'testData/8.60_A_30s_1100ms_3/',
    'testData/9.60_A_5s_600ms_3/',
    'testData/10.60_A_20s_600ms_3/',
    'testData/11.60_A_35s_600ms_3/',
    'testData/12.60_A_50s_600ms_3/',
    'testData/13.60_A_30s_600ms_2/',
    'testData/14.60_A_30s_600ms_4/',
    'testData/15.60_A_30s_600ms_6/',
    'testData/16.60_A_30s_600ms_8/',
    'testData/17.60_A_30s_600ms_3/',
    'testData/18.60_A+NA+AZA_30s_600ms_3/',
    'testData/19.60_A+NA_30s_600ms_3/',
    'testData/20.60_A+AZA_30s_600ms_3/',
]
file_paths2 = [
    #'testData/21.10_A+NA+AZA_30s_600ms_3/',
    'testData/22.130_A+NA+AZA_30s_600ms_3/',
    #'testData/23.60_A+NA+AZA_30s_200ms_3/',
    #'testData/24.60_A+NA+AZA_30s_1100ms_3/',
    #'testData/25.60_A+NA+AZA_5s_600ms_3/',
    #'testData/26.60_A+NA+AZA_50s_600ms_3/',
    #'testData/27.60_A+NA+AZA_30s_600ms_2/',
    #'testData/28.60_A+NA+AZA_30s_600ms_8/',
]

labels = [ _[12:-1] if _ [11] == '.' else _[11:-1] for _ in file_paths ]
labels2 = [ _[12:-1] if _ [11] == '.' else _[11:-1] for _ in file_paths2 ]

# XBG predict parameter (Robiłem w ramach testu czy coś wychodzi)

# Predict user class with various methods

In [ ]:
def multiClass(path, param,method):
    start_time = time.time()
    file_path = f'{path}/stockApp_merged_data.csv'
    data = pd.read_csv(file_path)
    classes= sorted(data[param].unique())
    mapping = {url: i for i, url in enumerate(classes)}
    data['mapped'] = data[param].map(mapping)
    if param != 'endpoint_url':
      urls = sorted(data['endpoint_url'].unique())
      url_mapping = {url: i for i, url in enumerate(urls)}
      data['endpoint_url_mapped'] = data['endpoint_url'].map(url_mapping)
      data = data[['mapped', 'endpoint_url_mapped', 'application_time', 'database_time', 'api_time', 'cpu_usage_db_test', 'cpu_usage_db', 'cpu_usage_web']]
    if param == 'endpoint_url':
      data = data[['mapped', 'application_time', 'database_time', 'api_time', 'cpu_usage_db_test', 'cpu_usage_db', 'cpu_usage_web']]
    X = data.drop(columns=['mapped'])
    y = data['mapped']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42,stratify=y)
    if method == 'ExtraTreesClassifier':
      model = ExtraTreesClassifier(n_estimators=1000, random_state=42, criterion='gini', max_depth=10, min_samples_split=4, max_features='sqrt', bootstrap=False) 
    if method == 'DecisionTreeClassifier':
      model = DecisionTreeClassifier(random_state=42, max_depth=8, criterion='gini', max_features='sqrt', min_samples_split=8, splitter='best', ccp_alpha=0)
    if method == 'KNeighborsClassifier':
      model = KNeighborsClassifier(n_neighbors=7, algorithm='auto', leaf_size=20, weights='uniform')
    if method == 'LinearSVC':
      model = LinearSVC(multi_class='ovr', class_weight='balanced', max_iter=10000, C=10,loss='squared_hinge', penalty='l1', random_state=42)
      sc=StandardScaler()
      scaler = sc.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)
    if method == 'MLPClassifier':
      model = MLPClassifier(random_state=42, alpha=0.0001, learning_rate='constant' , max_iter=200, batch_size=16, hidden_layer_sizes=(120,80,40), solver='adam', activation='relu')
      sc=StandardScaler()
      scaler = sc.fit(X_train)
      X_train = scaler.transform(X_train)
      X_test = scaler.transform(X_test)
    if method == 'RandomForestClassifier':
      model = RandomForestClassifier(max_depth=8, random_state=42, criterion='gini', n_estimators=500, min_samples_split=8, max_features='sqrt', bootstrap=False)
    if method == 'Xgb':
      model = xgb.XGBClassifier(booster = 'gbtree',objective='multi:softmax',num_class=len(classes),random_state = 42)
      param_grid = {
        'reg_lambda':[0,0.3,1],
        'gamma':[0,1,2],
        'grow_policy':['depthwise', 'lossguide'],
        'learning_rate':[0.05, 0.3, 0.6], 
        'max_depth':[8,10,12],
        'n_estimators':[100,200,300]
      }
      #grid = GridSearchCV(model, param_grid, cv=2)
      #grid.fit(X_train, y_train)
      #print(grid.best_estimator_)
      #print(grid.best_params_)
      #print(grid.best_score_)  
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    #accuracy = accuracy_score(y_test, y_pred)
    #f1 = f1_score(y_test, y_pred, average='weighted')
    print(f'{method} time: {time.time() - start_time}')
    #print(f'{method} - Accuracy: {accuracy}')
    #print(f'{method} - F1 Score: {f1}')
    print(classification_report(y_test,y_pred))
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=mapping.keys())
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix | {method}')
    plt.xticks(rotation=45, ha='right')
    plt.show()
    plt.close()
i=0    
for test_dir in file_paths2:
  print(labels2[i])
  methods = ['Xgb', 'ExtraTreesClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier', 'RandomForestClassifier', 'LinearSVC','MLPClassifier']
  for method in methods:
    multiClass(test_dir, 'user_class',method)
  i += 1